<a href="https://colab.research.google.com/github/dmitriia/mbs/blob/master/cuda/cuda_colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import codecs
import sys
import os 
import pandas as pd
import numpy as np
import pickle
import numba

In [0]:
# 1 ШАГ. Установка Пакетов для работы с GPU (После перезапуска, не выполнять)
!apt-get install nvidia-cuda-toolkit > /dev/null 2>&1
!pip install cudf > /dev/null 2>&1
!pip3 install cudf-cuda100 > /dev/null 2>&1
print("Next")

Next


In [0]:
# 2 Шаг. Проверка
import os
os.environ['NUMBAPRO_LIBDEVICE'] = "/usr/lib/nvidia-cuda-toolkit/libdevice"
os.environ['NUMBAPRO_NVVM'] = "/usr/lib/x86_64-linux-gnu/libnvvm.so.3"

from numba import cuda
import numpy as np
import time

@cuda.jit
def hello(data):
    data[cuda.blockIdx.x, cuda.threadIdx.x] = cuda.blockIdx.x

numBlocks = 5
threadsPerBlock = 10

data = np.ones((numBlocks, threadsPerBlock), dtype=np.uint8)

hello[numBlocks, threadsPerBlock](data)

print(data)

[[0 0 0 0 0 0 0 0 0 0]
 [1 1 1 1 1 1 1 1 1 1]
 [2 2 2 2 2 2 2 2 2 2]
 [3 3 3 3 3 3 3 3 3 3]
 [4 4 4 4 4 4 4 4 4 4]]


In [0]:
# 3 Шаг. Перезапустить: вкладка RUNTIME -> restart Runtime... -> Yes

In [0]:
!pip install cupy > /dev/null 2>&1

In [0]:
import cupy as cp

In [0]:
!pip install cffi 

In [0]:
!pip install pymssql
import pymssql

     |████████████████████████████████| 1.4MB 8.4MB/s 


In [0]:
lld_d = pd.read_csv("lld.csv")
lld_d.head()

,Unnamed: 0,Кол-во закладных,Суммарный ОД,Первоначальный ОД,Текущий ОД,Ставка по кредиту,Выдержка,Срок до погашения,Платеж,Кредит/Залог,Среднерыночная ставка при выдаче,Поправка дефолта,Поправка досрочки,Поправка частички 0,Поправка частички 1,Изменение стоимости залога,Продукт
0,0,169,8.071962e+05,1.722503e+06,8.071962e+05,10.591861,61.475589,56.260188,20460.725026,51.557590,12.686301,0.420820,1.243603,1.376291,0.994696,0.912645,4
1,1,141,9.538143e+05,1.477888e+06,9.538143e+05,14.230518,69.058343,116.764973,15332.340603,56.160008,12.454338,1.242220,1.190745,1.286317,0.970541,0.953632,4
2,2,167,1.104837e+06,1.432819e+06,1.104837e+06,10.578382,34.807412,143.052107,13688.347792,58.144696,12.447371,0.534198,1.271225,1.130885,1.010540,0.976501,4
3,3,161,1.188801e+06,1.614943e+06,1.188801e+06,14.736762,44.163368,192.975824,16143.838950,56.080902,13.022544,1.180361,1.264420,1.333765,1.014849,0.936259,4
4,4,85,3.009323e+06,3.399544e+06,3.009323e+06,10.433306,23.912431,215.627010,30975.545984,57.810672,11.005933,0.542988,1.196449,1.382037,0.971142,0.998581,4


In [0]:
scr_d = pd.read_csv("scr.csv")
scr_d[scr_d.columns[1]] = pd.to_datetime(scr_d[scr_d.columns[1]])
scr_d.head()

,Сценарий,Отчетная дата,"Кривая бескупонной доходности (zcyc, годовое начисление)","Сценарная доходность (yield curve, годовое начисление)","Сценарная короткая ставка (shortrate, годовое начисление)",Сценарная ставка по ипотеке,Сценарная ставка Моспрайм6m,Сценарный рост цен на недвижимость,Сценарный индекс цен на недвижимость,Сценарная ставка с лагом 6 мес.,Сезонный фактор,Сценарное дисконтирование,Рыночное дисконтирование
0,0,2019-10-01,0.782854,0.782854,0.782854,9.680000,7.153043,1.001531,1.001531,7.123266,10.0,1.000000,1.000000
1,0,2019-11-01,0.786975,0.942682,0.942682,9.166125,3.973045,1.004490,1.006028,7.142933,11.0,0.999218,0.999347
2,0,2019-12-01,0.790613,0.932785,0.922889,8.702453,2.409778,1.008736,1.014816,6.881182,12.0,0.998454,0.998688
3,0,2020-01-01,0.793785,0.884934,0.789299,8.284394,1.549598,1.009936,1.024899,6.737166,1.0,0.997800,0.998025
4,0,2020-02-01,0.796541,0.915444,1.007032,7.907554,1.657770,1.006927,1.031999,6.658528,2.0,0.996967,0.997359


In [0]:
lld_df = lld_d.set_index(lld_d.columns[0])
scr_df = scr_d.set_index([scr_d.columns[0], scr_d.columns[1]])

In [0]:
with open("super.pickle", "rb") as f:
  pick_module = pickle.load(f)
  

In [0]:
!unzip concept3.zip;
print( "end")

In [0]:
import specs
from importlib import reload
pick_module.Adjusts = specs.prime_m1
pick_module.Parameters.NumberOfMonteCarloScenarios = 15000
pick_module.Parameters.NumberOfSyntheticLoans = 60
pick_module.Parameters.NumberOfEventSimulations = 1000


In [0]:
reload(model.cfm)
import model
def do():
  # df = None
  df = []
  for i in range(10000):#(pick_module.Parameters.NumberOfMonteCarloScenarios):
    res = model.cfm.run(pick_module, lld_df, scr_df)
    #df.append(res)
  print(1)
  return df
    # df = pd.concat((res, df))
%timeit do()

1


In [0]:
#A - 2d, B - 1d
@cuda.jit(device=True)
def set_1d_to_2d(A_2d, B_1d, index):
  if(index < 0):
    for i in range(A_2d.shape[0]):
      for j in range(A_2d.shape[1]):
        A_2d[i, j] = B_1d[j]  
  else:
    for i in range(B_1d.shape[0]):
      A_2d[index, i] = B_1d[i]

@cuda.jit(device=True)
def set_2d_to_2d(A_2d, B_2d):
  for i in range(A_2d.shape[0]):
    for j in range(A_2d.shape[1]):
      A_2d[i, j] = B_2d[i,j]  

@cuda.jit(device=True)
def set_1d_to_3d(A_3d, B_1d, index):
  for i in range(A_3d.shape[1]):
    for j in range(A_3d.shape[2]):
      A_3d[index, i, j] = B_1d[j]  

@cuda.jit(device=True)
def set_2d_to_3d(A_3d, B_2d, index):
  for i in range(A_3d.shape[1]):
    for j in range(A_3d.shape[2]):
      A_3d[index, i, j] = B_2d[i,j] 

@cuda.jit(device=True)
def log_and_2d(A_2d, B_2d, result):
  for i in range(A_2d.shape[0]):
    for j in range(A_2d.shape[1]):
      result[i,j] = operator.gt(A_2d[i,j], B_2d[i,j])

@cuda.jit(device=True)
def gt_0(A_2d, result):
  for i in range(A_2d.shape[0]):
    for j in range(A_2d.shape[1]):
      result[i,j] = operator.gt(A_2d[i,j], 0)

@cuda.jit(device=True)
def gt_0_3d(A_3d):
  for i in range(A_3d.shape[0]):
    for j in range(A_3d.shape[1]):
      for k in range(A_3d.shape[1]):
        A_3d[i,j,k] = operator.gt(A_3d[i,j,k], 0)

@cuda.jit(device=True)
def get_2d_from_3d(A, ind, res):
  for i in range(A.shape[1]):
    for j in range(A.shape[2]):
      res[i,j] = A[ind, i, j]        


@cuda.jit(device=True)
def mul_1d_to_2d(A_2d, B_1d, res):
  for i in range(A_2d.shape[0]):
    for j in range(A_2d.shape[1]):
      res[i,j] = A_2d * B_1d[j]

@cuda.jit(device=True)
def multValue(A, v, result):
  for i in range(len(A)):
    result[i] = A[i] * v


@cuda.jit(device=True)
def mult_1d_1d(A_1d, B_1d, result):
  for i in range(A_1d.shape[0]):
    result[i] = A_1d[i] * B_1d[i]

@cuda.jit(device=True)
def mult_2d_1d(A_2d, B_1d, result):
  for i in range(A_2d.shape[0]):
    for j in range(A_2d.shape[1]):
      result[i,j] = A_2d[i, j] * B_1d[j]

@cuda.jit(device=True)
def sub_2d_2d(A_2d, B_2d, result):
  for i in range(A_2d.shape[0]):
    for j in range(A_2d.shape[1]):
      result[i,j] = A_2d[i, j] - B_2d[i,j]

@cuda.jit(device=True)
def div_2d_2d(A_2d, B_2d):
  for i in range(A_2d.shape[0]):
    for j in range(A_2d.shape[1]):
      if(B_2d[i,j] != 0):
        A_2d[i,j] = A_2d[i, j]/B_2d[i,j]

@cuda.jit(device=True)
def iand_2d_2d(A_2d, B_2d):
  for i in range(A_2d.shape[0]):
    for j in range(A_2d.shape[1]):
      A_2d[i,j] = operator.and_(A_2d[i,j], B_2d[i,j])

@cuda.jit(device=True)
def masked_2d_2d(A_2d, B_2d, result):
  for i in range(A_2d.shape[0]):
    for j in range(A_2d.shape[1]):
      if(B_2d[i,j]==1):
        result[i,j] = A_2d[i,j]
      else:
        result[i,j] = 0

@cuda.jit(device=True)
def any_2d(A_2d, result):
  result = True
  for i in range(A_2d.shape[0]):
    for j in range(A_2d.shape[1]):
      if(A_2d[i,j] != 0):
        break
    if(A_2d[i,j] != 0):
      break
    result = False

@cuda.jit(device=True)
def mask_len(A_2d):
  res = 0
  for i in range(A_2d.shape[0]):
    for j in range(A_2d.shape[1]):
      if (A_2d[i,j] == 1):
        res += 1
  return res

@cuda.jit(device=True)
def apply_mask(A_2d, mask_2d, result):
  # res = cuda.local.array(mask_len(mask_2d), dtype=numba.float64)
  for i in range(A_2d.shape[0]):
    for j in range(A_2d.shape[1]):
      if(mask_2d[i,j] == 1):
        result[i,j] = A_2d[i,j]
      else:
        result[i,j] = 0

@cuda.jit(device=True)
def imul_masked(A_2d, mask_2d, B_2d):
  for i in range(A_2d.shape[0]):
    for j in range(A_2d.shape[1]):
      if(mask_2d[i,j] == 1):
        A_2d[i,j] *= B_2d[i,j]

@cuda.jit(device=True)
def mean_3d_t(A_3d, result):
  # res = cuda.local.array((A_3d.shape[0], A_3d.shape[2]), dtype=numba.float64)
  for i in range(A_3d.shape[0]):
    for k in range(A_3d.shape[2]):
      sum = 0
      for j in range(A_3d.shape[1]):
        sum += A_3d[i,j,k]
      result[k,i] = sum / A_3d.shape[1]

@cuda.jit(device=True)
def min_2d_2d(A_2d, B_2d, result):
  for i in range(A_2d.shape[0]):
    for j in range(A_2d.shape[1]):
      result[i,j] = min(A_2d[i,j], B_2d[i,j])

@cuda.jit(device=True)
def merge(A_1, A_2, A_3, A_4, A_5, result):
  for i in range(A_2d.shape[0]):
    for j in range(A_2d.shape[1]):
      result[i,j] = min(A_2d[i,j], B_2d[i,j])

In [0]:
import operator
@cuda.jit
def dd2(debt, irate, payment, dt, r2, T, Debt_BoP,Debt_Int,Debt_Amt,Debt_Pmt,Plan_EoP, L0,AP, IRP, PR, result, res, ij):
  start = cuda.grid(1)
  stride = cuda.gridsize(1)

  # size_T1_S_N = (size_3[0], size_3[1], size_3[2],)
  # size_S_N = size_2[0], size_2[1]
  # size_1_N = (size_1[0], size_1[1],)
  # result = cuda.local.array(size_S_N, dtype=numba.float64)

  # Debt_BoP = cuda.local.array(size_T1_S_N, dtype=numba.float64)
  # Debt_Int = cuda.local.array(size_T1_S_N, dtype=numba.float64)
  # Debt_Amt = cuda.local.array((1000,100), dtype=numba.float64)
  # Debt_Pmt = cuda.local.array(size_T1_S_N, dtype=numba.float64)
  # Plan_EoP = cuda.local.array(size_S_N, dtype=numba.float64)
  # L0 = cuda.local.array(size_S_N, dtype=numba.float64)
  # AP = cuda.local.array(size_S_N, dtype=numba.float64)  # installment
  # IRP = cuda.local.array(size_1_N, dtype=numba.float64)  # interest rate

  set_1d_to_3d(Debt_BoP, debt, 0)

  multValue(payment, dt * 12 , result[0])
  set_1d_to_2d(AP, result[0], -1)

  multValue(irate, dt/100 , result[0])
  set_1d_to_2d(IRP, result[0], -1)


  for t in range(T):
    mult_2d_1d(Debt_BoP[t], IRP[0], Debt_Int[t])
    # set_2d_to_3d(Debt_Int, result, t)

    sub_2d_2d(AP, Debt_Int[t], result)
    min_2d_2d(result, Debt_BoP[t], Debt_Amt[t])
    # set_1d_to_3d(Debt_Amt, result, t)

    sub_2d_2d(Debt_BoP[t], Debt_Amt[t], Plan_EoP)
    # set_2d_to_2d(Plan_EoP, result)

    min_2d_2d(PR[t], Plan_EoP, Debt_Amt[t])
    # set_1d_to_3d(Debt_Amt, result, result)

    sub_2d_2d(Plan_EoP, Debt_Pmt[t], Debt_BoP[t + 1])
  #   set_2d_to_2d(Debt_BoP[t + 1], result)

    flag= True
    any_2d(Debt_BoP[t + 1], flag)

    if(flag):
      gt_0(Debt_BoP[t + 1], L0)
      # set_2d_to_2d(L0, result)
      gt_0(Debt_Pmt[t], result)
      # r2[t, 0,0] = True
      log_and_2d(r2[t], result, result)
      log_and_2d(L0, result, L0)
      ln = mask_len(result)
      apply_mask(Debt_BoP[t + 1], L0, result)
      result2 = result
      apply_mask(Plan_EoP, L0, result2)

      div_2d_2d(result, result2)

      imul_masked(AP, L0, result)
  
  mean_3d_t(Debt_BoP, res)
  gt_0_3d(Debt_BoP)
  mean_3d_t(Debt_BoP, res)
  mean_3d_t(Debt_Int, res)
  mean_3d_t(Debt_Amt, res)
  mean_3d_t(Debt_Pmt, res)
  ij[start]=1

  # NUM = mean_and_transpose(Debt_BoP)
  # YLD = mean_and_transpose(Debt_Int)
  # AMT = mean_and_transpose(Debt_Amt)
  # PMT = mean_and_transpose(Debt_Pmt)


In [0]:
import operator
size_3 = np.array((242,2000,90))
size_2 = np.array((2000,90), dtype = np.int64)
size_4 = (242,90)
size1 = np.array((1,90))

debt = np.array([i for i in range(90)]) # 2,3 debt

irate = np.array([i/10 for i in range(90)]) #3, 1 
# b = np.array([2,2,2]) #3, 1 irate

payment = np.array([i*100 for i in range(90)])

dt = 1
# r2 = cuda.to_device(np.array([[[True,False,True], [True,False,True], [True,False,True]],
#                               [[True,False,True], [True,False,True], [True,False,True]],
#                               [[True,False,True], [True,False,True], [True,False,True]]], dtype=bool))
r2 = cuda.to_device(np.random.random(size_3) < 2)
size_T1_S_N = size_3
size_S_N = size_2

Debt_BoP = cuda.to_device(np.zeros(size_T1_S_N, dtype=np.float64) ) # debt at the beginning of a period
Debt_Int = cuda.to_device(np.zeros(size_T1_S_N, dtype=np.float64) ) # payed interest
Debt_Amt = cuda.to_device(np.zeros(size_T1_S_N, dtype=np.float64) ) # debt amortization by plan
Debt_Pmt = cuda.to_device(np.zeros(size_T1_S_N, dtype=np.float64) ) # debt amortization by part prepayment        
Plan_EoP = cuda.to_device(np.zeros(size_S_N, dtype=np.float64))

L0 = cuda.to_device(np.zeros(size_S_N, dtype=np.bool) )

AP = cuda.to_device(np.zeros(size_S_N, dtype=np.float64) )
IRP = cuda.to_device(np.zeros(size1, dtype=np.float64) )

result = cuda.to_device(np.zeros(size_S_N, dtype=np.float64) )
result_bool = cuda.to_device(np.zeros(size_4, dtype=np.float64) )

PR = cuda.to_device(np.random.random(size_T1_S_N))
T = 12

threadsperblock = 32
blockspergrid = (256 + (threadsperblock - 1)) // threadsperblock
print(blockspergrid)
# increment_by_one[blockspergrid, threadsperblock](an_array)

# print(Debt_Int.copy_to_host())

# print(result.copy_to_host())  

# AP, Debt_Int[t]
# print(r2[0].copy_to_host())
# print(result_bool.copy_to_host())


8


In [0]:
threads = 128
# blocks = int(2496/128)#128
blocks = int(2496/128)#128


In [0]:
ij = cuda.to_device(np.zeros(threads * blocks))

#threads = 128
#blocks = 256#int(30000/threads)+1

%time dd2[blocks, threads](debt, irate, payment, dt, r2, T, Debt_BoP,Debt_Int,Debt_Amt,Debt_Pmt,Plan_EoP, L0, AP, IRP, PR, result, result_bool, ij)
# print(AP.copy_to_host())
aasd = ij.copy_to_host()
aasd1 = aasd[aasd == 1]
print(len(aasd1))

CPU times: user 1min 30s, sys: 1min 22s, total: 2min 52s
Wall time: 2min 53s
2432


In [0]:
def do_it(num):    
  for i in range(num):
    Run(debt, irate,  payment, size_T1_S_N, size_S_N, r2, IRP,AP, dt,T,PR)
%time do_it(threads * blocks)
print (threads * blocks)

CPU times: user 24min 43s, sys: 30.8 s, total: 25min 14s
Wall time: 25min 15s
2560


In [0]:
def Run(debt, irate, payment, size_T1_S_N, size_S_N, r2, IRP,AP, dt,T,PR):
  Debt_BoP = np.zeros(size_T1_S_N, dtype=np.float64)  # debt at the beginning of a period
  Debt_Int = np.zeros(size_T1_S_N, dtype=np.float64)  # payed interest
  Debt_Amt = np.zeros(size_T1_S_N, dtype=np.float64)  # debt amortization by plan
  Debt_Pmt = np.zeros(size_T1_S_N, dtype=np.float64)  # debt amortization by part prepayment        
  Plan_EoP = np.zeros(size_S_N, dtype=np.float64)

  # t0 = time.clock()

  # average debt size in bucket
  Debt_BoP[0] = debt.reshape((1, -1))
  # weighted average interest percent in bucket
  IRP[:, :] = irate.reshape((1, -1)) / 100.0 * dt
  # average payment amount in bucket
  AP[:, :] = payment.reshape((1, -1)) * 12.0 * dt

  # print('SR init: {0:.3f} sec'.format(time.clock() - t0)); t0 = time.clock()


  for t in range(T):

      # partial prepayment rate
      # self.PrepaymentFunction(t, self.pp_value)
      # self.DynamicPrepaymentRate(t)

      # payed interests
      Debt_Int[t] = Debt_BoP[t] * IRP

      # amortization by plan
      Debt_Amt[t] = np.minimum(AP - Debt_Int[t], Debt_BoP[t])

      # was:
      # self.L0[:, :] = Debt_BoP[t] <= Debt_Amt[t]
      # Debt_Amt[t][self.L0] = Debt_BoP[t][self.L0]

      # general debt at the end of the period planned, i.e. before unscheduled partial prepayment
      Plan_EoP[:, :] = Debt_BoP[t] - Debt_Amt[t]

      # partial prepayment - only for normal amortization
      Debt_Pmt[t] = np.minimum(PR[t], Plan_EoP)
      # # for z-spread
      # Debt_Pmt[t] = 0.017344417 * Debt_BoP[t]

      # was:
      # self.L0[:, :] = Plan_EoP <= Debt_Pmt[t]
      # Debt_Pmt[t][self.L0] = Plan_EoP[self.L0]

      # general debt at the end of period t, which is start of period t + 1
      Debt_BoP[t + 1] = Plan_EoP - Debt_Pmt[t]
      if not np.any(Debt_BoP[t + 1]):
          break

      # dynamic payment amount
      # self.AP[Debt_BoP[t] == 0] = 0.0            
      L0[:, :] = Debt_BoP[t + 1] > 0
      L0[:, :] &= r2[t] & (Debt_Pmt[t] > 0)
      AP[L0] *= (Debt_BoP[t + 1][L0] / Plan_EoP[L0])  # (Debt_BoP[t][self.L0] + Debt_Pmt[t - 1][self.L0]))

  # print('SR cycle: {0:.3f} sec'.format(time.clock() - t0))

  # # save i-th bucket history
  # self.NUM[:, :] = (Debt_BoP > 0.0).mean(axis=1).T
  # self.DBT[:, :] = (Debt_BoP).mean(axis=1).T
  # self.YLD[:, :] = (Debt_Int).mean(axis=1).T
  # self.AMT[:, :] = (Debt_Amt).mean(axis=1).T
  # self.PMT[:, :] = (Debt_Pmt).mean(axis=1).T

  # save i-th bucket history
  NUM = (Debt_BoP > 0.0).mean(axis=1).T
  DBT = (Debt_BoP).mean(axis=1).T
  YLD = (Debt_Int).mean(axis=1).T
  AMT = (Debt_Amt).mean(axis=1).T
  PMT = (Debt_Pmt).mean(axis=1).T